In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark import SparkContext
sc = SparkContext("local","test")
spark = SparkSession(sc)

22/08/07 23:10:30 WARN Utils: Your hostname, rohit resolves to a loopback address: 127.0.1.1; using 192.168.64.6 instead (on interface enp0s6)
22/08/07 23:10:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/08/07 23:10:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/08/07 23:10:32 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/08/07 23:10:32 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/08/07 23:10:32 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [3]:
df = spark.read.csv("hdfs://localhost:9000/user/rohit/scpca/2022-06-10.csv", inferSchema = True, header = True)
df.show()

+----------+--------+-------+---------+------+----------+-----------+-------+-------+-----+
|      date|    time|   size|r_version|r_arch|      r_os|    package|version|country|ip_id|
+----------+--------+-------+---------+------+----------+-----------+-------+-------+-----+
|2022-06-10|18:49:20|  18989|       NA|    NA|        NA|       jpeg|  0.1-9|     US|    1|
|2022-06-10|18:47:06| 288909|       NA|    NA|        NA|     dtplyr|  1.2.1|     US|    2|
|2022-06-10|18:49:18|2450682|       NA|    NA|        NA|        rgl|0.109.2|     GB|    3|
|2022-06-10|18:49:18| 425217|    3.5.2|x86_64| linux-gnu|       ragg|  1.2.2|     GB|    4|
|2022-06-10|18:49:24|  18390|       NA|    NA|        NA|    memoise|  2.0.1|     US|    5|
|2022-06-10|18:49:21| 342889|       NA|    NA|        NA|    usethis|  2.1.6|     US|    5|
|2022-06-10|18:49:12| 943544|       NA|    NA|        NA|    threejs|  0.3.3|     NA|    6|
|2022-06-10|18:49:21|2581945|    4.2.0|x86_64|darwin17.0|    ggiraph|  0.8.2|   

In [4]:
df1 = spark.read.csv("hdfs://localhost:9000/user/rohit/scpca/2022-06-10.csv", inferSchema = True, header = True, nullValue="NA")
df1.show()

+----------+--------+-------+---------+------+----------+-----------+-------+-------+-----+
|      date|    time|   size|r_version|r_arch|      r_os|    package|version|country|ip_id|
+----------+--------+-------+---------+------+----------+-----------+-------+-------+-----+
|2022-06-10|18:49:20|  18989|     null|  null|      null|       jpeg|  0.1-9|     US|    1|
|2022-06-10|18:47:06| 288909|     null|  null|      null|     dtplyr|  1.2.1|     US|    2|
|2022-06-10|18:49:18|2450682|     null|  null|      null|        rgl|0.109.2|     GB|    3|
|2022-06-10|18:49:18| 425217|    3.5.2|x86_64| linux-gnu|       ragg|  1.2.2|     GB|    4|
|2022-06-10|18:49:24|  18390|     null|  null|      null|    memoise|  2.0.1|     US|    5|
|2022-06-10|18:49:21| 342889|     null|  null|      null|    usethis|  2.1.6|     US|    5|
|2022-06-10|18:49:12| 943544|     null|  null|      null|    threejs|  0.3.3|   null|    6|
|2022-06-10|18:49:21|2581945|    4.2.0|x86_64|darwin17.0|    ggiraph|  0.8.2|   

In [5]:
r_version_with_most_counts = df1.filter('r_version is not null').groupBy('r_version').count().orderBy('count', ascending=False).collect()[0].r_version
df1 = df1.na.fill(value=r_version_with_most_counts,subset=["r_version"])

r_arch_with_most_counts = df1.filter('r_arch is not null').groupBy('r_arch').count().orderBy('count', ascending=False).collect()[0].r_arch
df1 = df1.na.fill(value=r_arch_with_most_counts,subset=["r_arch"])

r_os_with_most_counts = df1.filter('r_os is not null').groupBy('r_os').count().orderBy('count', ascending=False).collect()[0].r_os
df1 = df1.na.fill(value=r_os_with_most_counts,subset=["r_os"])

package_with_most_counts = df1.filter('package is not null').groupBy('package').count().orderBy('count', ascending=False).collect()[0].package
df1 = df1.na.fill(value=package_with_most_counts,subset=["package"])

version_with_most_counts = df1.filter('version is not null').groupBy('version').count().orderBy('count', ascending=False).collect()[0].version
df1 = df1.na.fill(value=version_with_most_counts,subset=["version"])

country_with_most_counts = df1.filter('country is not null').groupBy('country').count().orderBy('count', ascending=False).collect()[0].country
df1 = df1.na.fill(value=country_with_most_counts,subset=["country"])

df1.show()

+----------+--------+-------+---------+------+----------+-----------+-------+-------+-----+
|      date|    time|   size|r_version|r_arch|      r_os|    package|version|country|ip_id|
+----------+--------+-------+---------+------+----------+-----------+-------+-------+-----+
|2022-06-10|18:49:20|  18989|    4.2.0|x86_64|   mingw32|       jpeg|  0.1-9|     US|    1|
|2022-06-10|18:47:06| 288909|    4.2.0|x86_64|   mingw32|     dtplyr|  1.2.1|     US|    2|
|2022-06-10|18:49:18|2450682|    4.2.0|x86_64|   mingw32|        rgl|0.109.2|     GB|    3|
|2022-06-10|18:49:18| 425217|    3.5.2|x86_64| linux-gnu|       ragg|  1.2.2|     GB|    4|
|2022-06-10|18:49:24|  18390|    4.2.0|x86_64|   mingw32|    memoise|  2.0.1|     US|    5|
|2022-06-10|18:49:21| 342889|    4.2.0|x86_64|   mingw32|    usethis|  2.1.6|     US|    5|
|2022-06-10|18:49:12| 943544|    4.2.0|x86_64|   mingw32|    threejs|  0.3.3|     US|    6|
|2022-06-10|18:49:21|2581945|    4.2.0|x86_64|darwin17.0|    ggiraph|  0.8.2|   

In [6]:
df1.createOrReplaceTempView("packages")

In [7]:
question1 = spark.sql('SELECT COUNT(package) AS NUMBER_OF_ggplot2_DOWNLOADS FROM packages WHERE package="ggplot2"')
question1.show()

+---------------------------+
|NUMBER_OF_ggplot2_DOWNLOADS|
+---------------------------+
|                      96158|
+---------------------------+



In [8]:
question2 = spark.sql('SELECT COUNT(country) AS TOTAL_NUMBER_OF_PACKAGE_DOWNLOADS_FOR_US FROM packages WHERE country="US"')
question2.show()

+----------------------------------------+
|TOTAL_NUMBER_OF_PACKAGE_DOWNLOADS_FOR_US|
+----------------------------------------+
|                                 3603862|
+----------------------------------------+



In [9]:
question3 = spark.sql('SELECT package AS PACKAGE_NAME, max(size) AS SIZE FROM packages GROUP BY package order by max(size) desc')
question3.show(10)

+------------+---------+
|PACKAGE_NAME|     SIZE|
+------------+---------+
|         h2o|179368160|
|          sf|106868268|
|       terra|105924404|
|       rgdal| 98924892|
|   gdalcubes| 92972267|
|        Boom| 87584022|
|      vapour| 83397309|
|      mlpack| 53200349|
|      duckdb| 52988625|
|       torch| 44241470|
+------------+---------+
only showing top 10 rows



In [10]:
question4 = spark.sql('SELECT package AS PACKAGE, COUNT(package) AS NUMBER_OF_DOWNLOADS FROM packages GROUP BY package ORDER BY COUNT(package) DESC')
question4.show(10)

+--------+-------------------+
| PACKAGE|NUMBER_OF_DOWNLOADS|
+--------+-------------------+
| ggplot2|              96158|
|   rlang|              75117|
|   dplyr|              73894|
|     cli|              66245|
|devtools|              65148|
|   vctrs|              64526|
|    glue|              63487|
|      sf|              61552|
|     rgl|              55708|
|   rgeos|              55496|
+--------+-------------------+
only showing top 10 rows



In [11]:
question5 = spark.sql('SELECT DISTINCT(r_os) AS OS_USED_IN_DOWNLOADING_ggplot2 FROM packages WHERE package="ggplot2"')
question5.show()

+------------------------------+
|OS_USED_IN_DOWNLOADING_ggplot2|
+------------------------------+
|                  darwin15.6.0|
|                  darwin21.1.0|
|                  darwin21.3.0|
|                  darwin19.6.0|
|                  darwin19.4.0|
|                  darwin18.6.0|
|               linux-gnueabihf|
|                    linux-musl|
|                  darwin20.6.0|
|                    darwin17.0|
|                       mingw32|
|                  darwin13.4.0|
|                     linux-gnu|
|                      darwin20|
+------------------------------+



In [14]:
question6 = spark.sql('SELECT package, COUNT(package) AS NUMBER_OF_DOWNLOADS_IN_IRELAND FROM packages WHERE country="IR" GROUP BY package ORDER BY COUNT(package) DESC')
question6.show(10)

+--------+------------------------------+
| package|NUMBER_OF_DOWNLOADS_IN_IRELAND|
+--------+------------------------------+
|    glue|                            28|
|   rlang|                            28|
|   vctrs|                            24|
| ggplot2|                            24|
|magrittr|                            23|
|     cli|                            23|
|   RCurl|                            23|
|    Rcpp|                            21|
|  scales|                            21|
|  pillar|                            21|
+--------+------------------------------+
only showing top 10 rows



In [15]:
question7a = spark.sql('SELECT ip_id AS IP, COUNT(ip_id) AS NUMBER_OF_DOWNLOADS FROM packages GROUP BY ip_id ORDER BY COUNT(ip_id) DESC')
question7a.show(5)

question7b = spark.sql('SELECT DISTINCT(r_os) AS OS_USED_BY_IP_6 FROM packages WHERE ip_id = "6"')
question7b.show()

+----+-------------------+
|  IP|NUMBER_OF_DOWNLOADS|
+----+-------------------+
|   6|             542414|
|   4|             223731|
|1658|             138513|
|   8|              92689|
|   3|              84957|
+----+-------------------+
only showing top 5 rows



+---------------+
|OS_USED_BY_IP_6|
+---------------+
|   darwin15.6.0|
|   darwin21.1.0|
|   darwin20.2.0|
|   darwin21.3.0|
|   darwin19.6.0|
|   darwin20.4.0|
|   darwin18.6.0|
|   darwin21.5.0|
|   darwin20.3.0|
|linux-gnueabihf|
|   darwin18.7.0|
|     linux-musl|
|   darwin20.6.0|
|     darwin17.0|
|        mingw32|
|   darwin13.4.0|
|      linux-gnu|
|       darwin20|
+---------------+



In [21]:
question8 = spark.sql('SELECT r_os AS OS, COUNT(r_os) AS COUNT FROM packages GROUP BY r_os ORDER BY COUNT(r_os) DESC')
question8.show(5)

+------------+-------+
|          OS|  COUNT|
+------------+-------+
|     mingw32|4844271|
|   linux-gnu| 781997|
|  darwin17.0| 533670|
|    darwin20|  69036|
|darwin15.6.0|  25962|
+------------+-------+
only showing top 5 rows



In [17]:
question9 = spark.sql('SELECT COUNT(DISTINCT(ip_id)) AS NUMBER_OF_USERS_USING_32_BIT_MACHINES FROM packages WHERE r_arch = "i386"')
question9.show()

+-------------------------------------+
|NUMBER_OF_USERS_USING_32_BIT_MACHINES|
+-------------------------------------+
|                                 1028|
+-------------------------------------+



In [18]:
df.createOrReplaceTempView("packagesold")

In [20]:
question10 = spark.sql('Select COUNT(*) AS LINES_WITH_MISSING_VALUES FROM packagesold WHERE r_version = "NA" OR r_os="NA" OR r_arch="NA" OR package="NA" OR version="NA" OR country="NA"')
question10.show()

+-------------------------+
|LINES_WITH_MISSING_VALUES|
+-------------------------+
|                  3214567|
+-------------------------+



In [49]:
sc.stop()